In [3]:
from langchain import HuggingFaceHub
import pandas as pd
import json
from sentence_transformers import SentenceTransformer
from chromadb import Client
from chromadb.config import Settings
import os
from dotenv import load_dotenv                                 
from langchain_google_genai import GoogleGenerativeAI           

load_dotenv()
GOOGLE_GEMINI_API = os.getenv("GOOGLE_GEMINI_API")

llm = GoogleGenerativeAI(model="gemini-pro", api_key=GOOGLE_GEMINI_API)

data = pd.read_excel("Espace_Original_Batch1.xlsx")

file_path = "Espace_Original_Batch1.xlsx"  
data = pd.ExcelFile(file_path)


import pandas as pd
from sentence_transformers import SentenceTransformer
from chromadb import Client, Settings

file_path = "Espace_Original_Batch1.xlsx"
data = pd.ExcelFile(file_path)

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

chroma_client = Client(Settings(persist_directory="db"))

numerical_columns_query = [
    'total_quantite Calculated', 'poids_brut_total_pesee', 'montant_fret',
    'poids_net_kg', 'montant_total_facture', 'poids_net', 'quantite'
]

numerical_columns_item_details = [
    'qte_facturee', 'montant_facture_par_ligne', 'montant_unitaire_par_ligne_facture'
]  
collection_name = "export_query_data_with_numericals"
collection = chroma_client.get_or_create_collection(name=collection_name)

def process_query_sheet():
    print("Processing 'query' sheet...")
    sheet_data = data.parse('query')

    sheet_data['text'] = sheet_data[
        ['exportateur', 'expediteur', 'destinataire', 'adresse_expediteur', 'origine',
         'adresse_destinataire', 'devise', 'paiement', 'importateur', 'client',
         'conditions_livraison', 'accords']
    ].astype(str).agg(lambda x: " | ".join(x), axis=1)

    collection = chroma_client.get_or_create_collection(name="export_query_data_with_numericals")

    for idx, row in sheet_data.iterrows():
        text = row['text']
        metadata = {
            "id": f"query_{idx}",
            "exportateur": row.get('exportateur', None),
            "destinataire": row.get('destinataire', None),
            **{col: row[col] for col in numerical_columns_query if col in row}
        }
        embedding = embedding_model.encode(text).tolist()
        collection.add(documents=[text], metadatas=[metadata], ids=[str(metadata['id'])])

    print("'query' sheet processed successfully!")

def process_item_details_sheet():
    print("Processing 'item_details' sheet...")
    sheet_data = data.parse('items_details')  
    sheet_data['text'] = sheet_data[
        ["Nom",'ref_client / Part Number', 'designation_facture']
    ].astype(str).agg(lambda x: " | ".join(x), axis=1)
    
    collection = chroma_client.get_or_create_collection(name="items_details_data")
    
    for idx, row in sheet_data.iterrows():
        text = row['text']
        metadata = {
            "id": f"item_details_{idx}",
            "sheet_name": "items_details",
            "ref_client": row.get('ref_client / Part Number', ""),
            "designation_facture": row.get('designation_facture', ""),
 
        }
        
        embedding = embedding_model.encode(text).tolist()
        collection.add(documents=[text], metadatas=[metadata], ids=[metadata['id']])
    
    print("'item_details' sheet processed and stored in ChromaDB!")



process_query_sheet()
process_item_details_sheet()

def query_chromadb(question, top_k=5):

    question_embedding = embedding_model.encode(question).tolist()

    results = collection.query(query_embeddings=[question_embedding], n_results=top_k)
    return results  
def query_chromadb(question, top_k=5):
    question_embedding = embedding_model.encode(question).tolist()
    results = collection.query(query_embeddings=[question_embedding], n_results=top_k)
    return results    

print("All sheets processed and stored in ChromaDB!")

question = "What is the total invoice for IMEPSA MAROC ??"
results = query_chromadb(question)  
retrieved_context = "\n".join([f"Document: {doc}\nMetadata: {meta}" for doc, meta in zip(results['documents'], results['metadatas'])])
llm_prompt = f"Answer the question based on the following context:\n{retrieved_context}\n\nQuestion: {question}\n Please provide a detailed answer Answer:"

response = llm.invoke(llm_prompt)
print(response.strip())    

Processing 'query' sheet...
'query' sheet processed successfully!
Processing 'item_details' sheet...
'item_details' sheet processed and stored in ChromaDB!
All sheets processed and stored in ChromaDB!
The total invoice for IMEPSA MAROC is 2968 €.
